# Velocity and pumping diagnostics after PharaGlow analysis

## Changing names (user input required)

#### Strain directories

In [ ]:
# ADD DIRECTORY OF STRAIN MASTER FOLDER(S)
ctrl = "/media/scholz_la/hd2/Nicolina/Pharaglow/Old_files/Pharaglow_v5/10x_GRU101_RFP_24hr/"
INF100 = "/media/scholz_la/hd2/Nicolina/Pharaglow/Pharaglow_batch/INF100/"
INF102 = "/media/scholz_la/soma/Nicolina/INF102/RFP_24h/10x/10x_INF102_analyzed/"
INF103 = "/media/scholz_la/soma/Nicolina/INF103/RFP_24h/10x/10x_INF103_analyzed/"
INF5 = "/media/scholz_la/soma/Nicolina/INF5/RFP_24h/10x/10x_INF5_analyzed/"

# FINAL FIGURE NAME AND DIRECTORY (figure name MUST have .pdf at the end) - if not changed then the previous figure
# WILL be erased so be careful!
avgplot = "/home/nzjacic/Desktop/10x_diagnostics_average_{}.pdf"
singleplot = "/home/nzjacic/Desktop/10x_diagnostics_single_{}.pdf"

#### Dictionary linking strain names to their folders and graph colors

In [ ]:
# ADD STRAIN NAMES
strains = ['Control', 'INF100', 'INF102', 'INF103', 'INF5']

# ADD DIRECTORY VARIABLE
directories = [ctrl, INF100, INF102, INF103, INF5]

# ADD COLORS
# if a syntax error comes up it's probably because you forgot to add a comma after the previous strain name
colors = {
        'INF100':'red',
        'Control':'blue',
        'INF102' :'green',
        'INF103':'orange',
        'INF5':'purple'
    }

#### Variables

In [ ]:
# FPS?
fps = 30

# HOW MANY MICRONS PER PIXEL?
umPerPx = 2.34

# HOW LONG BEFORE AND AFTER ENTRY WORM ENTRY TO PLOT (IN FRAMES)
tBefore = 70*fps
tAfter = 10*fps

## Extracting velocity and pumping information

### Importing packages

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import os
import pickle
# image io and analysis
import json
import pims
import trackpy as tp

# plotting
import matplotlib  as mpl 
import matplotlib.pyplot as plt 

#our packages
from pharaglow import extract, util
from pharaglow.util import smooth

### Defining functions and dictionaries

#### Defining functions to extract velocity, pumping and kymo

In [ ]:
def getVelocity(traj, umperPx, fps):
    return np.sqrt((np.diff(traj['x'])**2+np.diff(traj['y'])**2))/np.diff(traj['frame'])*umperPx*fps

def getPumps(pumps, ws = 30, prs = np.linspace(0.5,0.95,50) ):
    return extract.bestMatchPeaks(pumps, ws, prs)

def getKymo(df, key):
    kymo = np.sum([np.array(list(filter(None.__ne__,row))) for row in df[key].values], axis=2)
    kymo = np.array([np.interp(np.linspace(0, len(row), 100), np.arange(len(row)), np.array(row)) \
                      for row in kymo])
    kymo = extract.alignKymos(kymo).T
    return kymo

#### Defining how Python reads in Pharaglow and lawn data

In [ ]:
def getVelocity(traj, umperPx, fps):
    return np.sqrt((np.diff(traj['x'])**2+np.diff(traj['y'])**2))/np.diff(traj['frame'])*umperPx*fps

def getPumps(pumps, ws = 30, prs = np.linspace(0.5,0.95,50) ):
    return extract.bestMatchPeaks(pumps, ws, prs)

def getKymo(df, key):
    kymo = np.sum([np.array(list(filter(None.__ne__,row))) for row in df[key].values], axis=2)
    kymo = np.array([np.interp(np.linspace(0, len(row), 100), np.arange(len(row)), np.array(row)) \
                      for row in kymo])
    kymo = extract.alignKymos(kymo).T
    return kymo

def readData(dataFolder, umPerPx, fps, j = 0, nmax = None):
    df = {}
    path = os.path.dirname(dataFolder)
    for fn in os.listdir(path):
        if nmax != None and len(df.keys())>nmax-1:
            break
        file = os.path.join(path,fn)
        if os.path.isfile(file) and 'results_' in fn and fn.endswith('.json'):
            print('Reading', file)
            particle_index = int(fn.split('.')[0].split('_')[-1])
            traj =  pd.read_json(file, orient='split', numpy = True)
            # velocity
            t = traj['frame']/fps
            v = getVelocity(traj, umPerPx, fps)
            # pumping related data
            kymo = getKymo(traj, 'Straightened')
            rawPump = [-np.max(np.std(sIm, axis =1), axis =0) for sIm in traj['Straightened']]
            traj['pump'] = rawPump
            prs =  np.linspace(0.15,1.00,50)
            p, pump, pks, roc, metric  = getPumps(traj['pump'].values, prs = prs)
            #plotPumpAnalysis(pump, pks, roc, prs, time = traj['frame'], metric=metric)
            pinterp = np.interp(traj['frame'], p[:-1]+traj['frame'].iloc[0], fps/np.diff(p))
            # get a binary trace where pumps are 1 and non-pumps are 0
            tmp = np.zeros(len(t))
            tmp[p] = 1
            
            df[j] = {'time': t.values,
                     'x': traj['x'].values,
                     'y': traj['y'].values,
                             'velocity':v,
                             'peaks': p.values,
                             'pumpTrace':pump,
                             'binaryPumps': tmp,
                             'pumpInterp': pinterp,
                             'inside': traj['inside'].values,
                             'insideF': traj['insideHeadIntensity'].values,
                             'pid':particle_index,
                             'filename': fn,
                             'kymo': kymo,
                             'fps': fps,
                             'scale':umPerPx
                            }
            j +=1
    return df

#### Defining entry time

In [ ]:
def zscore(x, window):
    r = x.rolling(window=window)
    m = r.mean().shift(1)
    s = r.std(ddof=0).shift(1)
    z = (x-m)/s
    return z

def getEntryTime(data, method = 'binary', window = 30):
    if method =='binary':
        # get the first entry from binarized lawn data. data should be the 'inside' variable from pharaglow
        t0 = np.where(data ==1)[0]
    if method =='fluorescence':
        data = pd.Series(data)
        data -= data.mean()
        data /= data.std()
        t0 = np.where(data>0)[0]
    if len(t0)>0:
       # plt.axvline(t0[0])
        return t0[0]
    else:
        return np.nan

### Helper functions to generate aligned velocity and pumping data
def alignData(df, tBefore, tAfter):
    """df is a dictionary created in readData. We will find the point of lawn entry and align and crop data to
    tBefore, TAfter. tBefore, tAfter are in frames."""
    for idx in df.keys():
        # identify the inside point t0 or set a flag that it doesn't enter
        try:
            # t0 is in frames not seconds!
            # CAN SWAP BETWEEN BINARY AND FLUORESCENT DETECTION OF LAWN ENTRY HERE
            t0 = getEntryTime(df[idx]['insideF'], method = 'fluorescence')
            #t0 = getEntryTime(df[idx]['inside'], method = 'binary')
            print(t0, np.mean(df[idx]['inside']))
            if t0 > tBefore and len(df[idx]['inside']) > t0+tAfter:
                # update the data
                df[idx]['t0'] = t0
                df[idx]['enter'] = True
            else:
                df[idx]['t0'] = None
                df[idx]['enter'] = False
        except IndexError:
            df[idx]['t0'] = None
            df[idx]['enter'] = False
        # calculate cropped versions of the data for animals that entered the lawn
        if df[idx]['enter']:
            for dname in ['insideF', 'velocity', 'time', 'pumpTrace', 'binaryPumps', 'pumpInterp']:
                df[idx][dname+'Cut'] = df[idx][dname][df[idx]['t0']-tBefore: df[idx]['t0']+tAfter+1]
            # peaks and other info
            df[idx]['peaksCut'] = df[idx]['peaks'][(df[idx]['peaks'] > df[idx]['t0']-tBefore)&
                                    (df[idx]['peaks'] < df[idx]['t0']+tAfter+1)]
            #shift the cut peaks to the correct locations relative to cut window
            df[idx]['peaksCut'] =  df[idx]['peaksCut'] - df[idx]['t0']+tBefore
            df[idx]['timeEntry'] = np.arange(-tBefore, tAfter+1)
            df[idx]['tBefore'] = tBefore
            df[idx]['tAfter'] = tAfter
    return df

#### Defining average plotting function

In [ ]:
def graphing(Data, condition, plotpath, method = 'single', save = 'Yes'):
    # setting up GridSpec
    f1 = plt.figure(figsize = (16,4))
    grid = mpl.gridspec.GridSpec(ncols=3, nrows = 1, hspace = 0.3)
    # creating subplots
    axa = f1.add_subplot(grid[0,0])
    axb = f1.add_subplot(grid[0,1])
    axc = f1.add_subplot(grid[0,2])
    # axes dictionary
    axes = {
        'xlabel':'Time(s)',
        'ylabelv':'Velocity (um/s)',
        'ylabelp':'Pumping rate (Hz)',
        'ylabelf' : 'Fluorescence'
    }
    # extracting necessary data
    t, v, pr, f = [], [], [], []
    for idx in Data.keys():
        if Data[idx]['enter']:
            # pumping rate using rolling mean rate in a rolling 1 second window
            tAfter = Data[idx]['tAfter'] 
            #tmp = pd.Series(df[idx]['binaryPumpsCut']).rolling(fps).sum()
            tmp = Data[idx]['pumpInterpCut']
            t.append(Data[idx]['timeEntry']/Data[idx]['fps'])
            pr.append(tmp)
            v.append(Data[idx]['velocityCut'])
            f.append(Data[idx]['insideFCut']/np.max(Data[idx]['insideFCut']))
    print ('Plotting', condition+'...')
    # adding titles
    if method == 'average':
        # average velocity across all trajectories and all movies per strain
        mv, sv = np.mean(np.array(v), axis =0), np.std(np.array(v), axis =0)
        time = np.mean(t, axis=0)
        axa.plot(time, mv, color = colors[condition], alpha = 0.7)
        axa.fill_between(time, mv-sv, mv+sv, 
                             color = 'gray', alpha = 0.3)
        #axa.axvline(x = Data[idx]['timeEntry']/Data[idx]['fps'], color = 'black', dashes = (5, 2))
        axa.axvspan( 0, tAfter/Data[idx]['fps'], color='k', alpha=0.3)
        # making it pretty
        axa.annotate('n = '+str(len(v)), (0.8, 0.8), xycoords='axes fraction')
        axa.set_ylabel(axes['ylabelv'])
        axa.set_xlabel(axes['xlabel'])
        axa.set_ylim(0,200)
        # average pumping rate across all trajectories and all movies per strain
        mp, sp = np.mean(np.array(pr), axis = 0), np.std(np.array(pr), axis =0)
        axb.plot(time, mp, color = colors[condition], alpha = 0.5)
        axb.fill_between(time, mp-sp, mp+sp, 
                 color = 'gray', alpha = 0.3)
        axb.plot(time, smooth(mp, 50), 'black', lw =2)
        #axb.axvline(x = Data[idx]['timeEntry']/Data[idx]['fps'], color = 'black', dashes = (5, 2))
        axb.axvspan( 0, tAfter/Data[idx]['fps'], color='k', alpha=0.3)
        # making it pretty
        axb.annotate('n = '+str(len(pr)), (0.8, 0.8), xycoords='axes fraction')
        axb.set_ylabel(axes['ylabelp'])
        axb.set_xlabel(axes['xlabel'])
        axb.set_ylim(0,6)
        # average fluorescence sanity check
        mf, sf = np.mean(np.array(f), axis =0), np.std(np.array(f), axis =0)
        axc.plot(time, mf, color = colors[condition], alpha = 0.7)
        axc.fill_between(time, mf-sf, mf+sf, 
                             color = 'gray', alpha = 0.3)
        #axa.axvline(x = Data[idx]['timeEntry']/Data[idx]['fps'], color = 'black', dashes = (5, 2))
        axc.axvspan( 0, tAfter/Data[idx]['fps'], color='k', alpha=0.3)
        # making it pretty
        axc.annotate('n = '+str(len(f)), (0.8, 0.8), xycoords='axes fraction')
        axc.set_ylabel(axes['ylabelf'])
        axc.set_xlabel(axes['xlabel'])
        axc.set_ylim(0,1)
        # setting axes titles
        axa.set_title('Average Velocity of Trajectories per Video')
        axb.set_title('Pumping Rate (Hz) of Trajectories per Video')
        axc.set_title('Average Fluorescence')
        # saving data variable
        plotpath = plotpath.format(condition)
        print('Ta da!')
        # OPTIONAL - save figure
        if save == 'Yes':
            print(plotpath)
            plt.savefig(plotpath)
        else:
            print('Not saved')
    if method == 'single':
        time = t
        # velocities across all trajectories and all movies per strain
        axa.plot(time, v, color = colors[condition], alpha = 0.1)
        axa.axvspan( 0, tAfter/Data[idx]['fps'], color='k', alpha=0.3)
        # making it pretty
        axa.annotate('n = '+str(len(f)), (0.8, 0.8), xycoords='axes fraction')
        axa.set_ylabel(axes['ylabelv'])
        axa.set_xlabel(axes['xlabel'])
        axa.set_ylim(0,250)
        # pumping rates across all trajectories and all movies per strain
        axb.plot(time, pr, color = colors[condition], alpha = 0.1)
        axb.annotate('n = '+str(len(f)), (0.8, 0.8), xycoords='axes fraction')
        axb.axvspan(0, tAfter/Data[idx]['fps'], color='k', alpha=0.3)
        # making it pretty
        axb.set_ylabel(axes['ylabelp'])
        axb.set_xlabel(axes['xlabel'])
        axb.set_ylim(0,6)
        # fluorescence sanity check
        axc.plot(time, f, color = colors[condition], alpha = 0.1)
        # making it pretty
        axc.set_ylabel(axes['ylabelf'])
        axc.set_xlabel(axes['xlabel'])
        axc.set_ylim(0,1)
        # setting axes titles
        axa.set_title('Velocity of all Trajectories per Video')
        axb.set_title('Pumping Rate (Hz) of Trajectories per Video')
        axc.set_title('Fluorescence')
        # saving data variable
        plotpath = plotpath.format(condition)
        print('Ta da!')
        # OPTIONAL - save figure
        if save == 'Yes':
            print(plotpath)
            plt.savefig(plotpath)
        else:
            print('Not saved')

#### Defining saving and loading process

In [ ]:
def readPharaglow(strains, directories, index, save=True):
    Data = readData(directories[index], umPerPx, fps, nmax = None)
    if save:
        saveData = open(f'/home/nzjacic/Desktop/Diagnostics/Saved_data/{strains[index]}', 'wb')
        pickle.dump(Data, saveData)
        saveData.close()

In [ ]:
def loadPharaglow(strains, directories, index):
    saveData = open(f'/home/nzjacic/Desktop/Diagnostics/Saved_data/{strains[index]}', 'rb')
    Data = pickle.load(saveData)
    saveData.close()
    return Data

### Reading and Graphing (add as strains increase)

#### Control

In [ ]:
#readPharaglow(strains, directories, index=0)

In [ ]:
Data = loadPharaglow(strains, directories, index=0)
Data = alignData(Data, tBefore=tBefore, tAfter=tAfter)
# change plot variable and method
graphing(Data, strains[0], singleplot, method = 'single', save = 'Yes')

In [ ]:
v, pr = [], []
for idx in Data.keys():
    if Data[idx]['enter']:
        # pumping rate using rolling mean rate in a rolling 1 second window
        #tmp = pd.Series(df[idx]['binaryPumpsCut']).rolling(fps).sum()
        tmp = Data[idx]['pumpInterpCut']
        pr.append(tmp)
        v.append(Data[idx]['velocityCut'])
plt.scatter(v, pr, color = 'blue', alpha = 0.4)
plt.ylabel('Pumping rate (Hz)')
plt.xlabel('Velocity (um/s)')

#### INF100

In [ ]:
#readPharaglow(strains, directories, index=1)

In [ ]:
Data = loadPharaglow(strains, directories, index=1)
Data = alignData(Data, tBefore=tBefore, tAfter=tAfter)
graphing(Data, strains[1], singleplot, method = 'single', save = 'Yes')

#### INF102

In [ ]:
#readPharaglow(strains, directories, index=2)

In [ ]:
Data = loadPharaglow(strains, directories, index=2)
Data = alignData(Data, tBefore=tBefore, tAfter=tAfter)
graphing(Data, strains[2], singleplot, method = 'single', save = 'Yes')

#### INF103

In [ ]:
#readPharaglow(strains, directories, index=3)

In [ ]:
Data = loadPharaglow(strains, directories, index=3)
Data = alignData(Data, tBefore=tBefore, tAfter=tAfter)
graphing(Data, strains[3], singleplot, method = 'single', save = 'Yes')

#### INF5

In [ ]:
#readPharaglow(strains, directories, index=4)

In [ ]:
Data = loadPharaglow(strains, directories, index=4)
Data = alignData(Data, tBefore=tBefore, tAfter=tAfter)
graphing(Data, strains[4], singleplot, method = 'single', save = 'Yes')